In [ ]:
import boto3
import os
import ctypes
import uuid
from sklearn.neighbors import NearestNeighbors
import pickle
import numpy as np

for d, _, files in os.walk('lib'):
    for f in files:
        if f.endswith('.a'):
            continue
        ctypes.cdll.LoadLibrary(os.path.join(d, f))

s3_client = boto3.client('s3')

#load names
bucket = 'azclassifier'
key = 'names.pkl'
download_path = '/tmp/{}{}'.format(uuid.uuid4(), key)
s3_client.download_file(bucket, key, download_path)

f = open(download_path, 'rb')
names = pickle.load(f)
f.close()

#load array
bucket = 'azclassifier'
key = 'np_array.pkl'
download_path = '/tmp/{}{}'.format(uuid.uuid4(), key)
s3_client.download_file(bucket, key, download_path)

f = open(download_path, 'rb')
np_array = pickle.load(f)
f.close()

neigh = NearestNeighbors(n_neighbors=5, metric = 'cosine')
neigh.fit(np_array)

def handler(event, context):
    
    #Info
    my_word = float(event.get('my_word')['array'])
    ind = names.index(my_word)
    neib = neigh.kneighbors(np_array[ind].reshape(1, -1))
    
    answer = []
    for i in neib[1][0]:
        answer.append(names[i])
    
    return answer